# Multi-Objective Resource Allocation Method for IoT applications in the context of Mobile Edge Computing (MEC)

## Dépendances
Import des dépendances Python nécessaires au projet.

In [1]:
import numpy as np

## Dataset
Cette partie décrit le dataset et les données utilisées pour répondre au problème décrit plus bas.

## <p style="color:red;">Partie à remplacer par le vrai dataset de Kaggle</p>

Données en fonction d’autres données :
- $TE_m=TM_m+TW_m+TB_m$ le temps total de complétion d’une tâche $atn_m$
- $TM_m$ 
- $TW_m$
- $TB_m$
- $msu_n$ la moyenne de l’utilisation du serveur $men_n$
- $ZYS$ le compte total des VMs occupées (tous serveurs)
- $MSU$ la variance de l’équilibre de charge des serveurs 
- $wnt_n$ le temps maximum de complétion d’une tâche par le serveur $men_n$
- $ECB$ la consommation d’énergie pour un serveur $men_n$ en fonctionnement
- $ECF$ la consommation pour toutes les VMs occupées
- $ECK$ la consommation pour toutes les VMs inoccupées
- $EC$ la consommation totale d’énergie par le set de serveurs $MEN$

#### Constantes pour le Load Balancing

In [ ]:
# taille des VMs découlent de la taille de la tâche.

# nombre de VMs par serveurs déjà exprimé dans la RAS

#### Constantes pour le temps de complétion des tâches
<p style="color:red;">Peut être à transformer en matrice pour que chaque tâche ait une taille différente...</p>

In [ ]:
# Taille de la tâche en unités de données
G = 5  # = RG = GR
wk = {0:0.75, 1:1.25, 2:0.3} # Bande passante selon le cas j = 0, 1, 2

#### Constantes pour la consommation d'énergie

In [ ]:
# Constantes pour la consommation d'énergie
rx = 0.2 # Wattage d'un serveur allumé
rf = 0.125 # Wattage d'une VM occupée
rn = 0.5 # Wattage d'une VM non occupée

### Matrices/Opérateurs
- Serveur $men_n$ occupé ou non (1/0) : $c(men_n)$
- Tâche $atn_n$ lancée sur le serveur $men_m$ ou non (1/0) : $\phi(atn_n)$

In [ ]:
# Opérateurs C, S et phi à implémenter
C = 0 # 1 si le serveur est occupé ou 0 sinon
phi = 0 # 1 si la vm tourne sur le serveur ou 0 sinon
S = 5

### Paramètres expérimentaux
>On considère qu'une application ne génère qu'une seule tâche. Le nombre de tâches est donc directement corellé avec le nombre d'applications IoT.

In [ ]:
# Paramètre de nombre d'applications IoT
nb_app = [50, 100, 150, 200, 250]
# Paramètre du nombre de VM par serveur MEC
nb_vm = 8
# Paramètre de la quantité de VM nécessaire par application
nb_vm_app = [1,7]
# Paramètre de la taille de la VM à exécuter
size_vm =

### RAS de départ

In [ ]:
# Wattage des serveurs
pr_serv = 350
# Wattage des VM occupées
pr_ovm = 75
# Wattage des VM inoccupées
pr_uvm = 45

## Définition du problème dans le code

### Définition des contraintes

In [ ]:
# def constraint1(x):
#     # Le nombre de vm obtenue par iot ne doit pas exceder le nombre de machines inactive sur le serveur
#     return np.array([np.sum(x[i::n_noeuds]) for i in range(n_taches)]) - 1
#
# def constraint2(x):
#     # La capacité de traitement maximale de chaque nœud ne doit pas être dépassée
#     return C - np.dot(w, x.reshape((n_taches, n_noeuds)))

#Contraintes

def c1_nbVM(position):
    """
    Vérifie si le nombre de VMs actives par serveur MEC est inférieur ou égal à M.
    Args:
        variables: Un tableau contenant les valeurs des variables de décision.

    Returns:
        True si la contrainte est satisfaite, False sinon.
    """
    # Décodage des variables
    x = position
    # Calcul du nombre de VMs actives par serveur MEC
    nb_vms_par_serveur = np.sum(x.reshape((12, 72)), axis=1)
    # Vérification de la contrainte
    return np.all(nb_vms_par_serveur <= M)

def c2_ressources(position):
    """
    Vérifie si la charge de chaque serveur MEC est inférieure ou égale à sa capacité maximale.

    Args:
        variables: Un tableau contenant les valeurs des variables de décision.

    Returns:
        True si la contrainte est satisfaite, False sinon.
    """
    # Décodage des variables
    x = position
    # Calcul de la charge de chaque serveur MEC
    charge_serveur = np.dot(x, sigma)
    # Vérification de la contrainte
    return np.all(charge_serveur <= capacites_serveurs[men])

### Définition des Fonctions Objectifs

In [ ]:
def f1_time(position):
  """
  Calcule le temps de complétion total des tâches.

  Args:
    position: Un tableau contenant les valeurs des variables de décision.

  Returns:
    Le temps de complétion total (TE).
  """

  # Décodage des variables
  x = position  # Affectation des variables

  # Calcul du temps de traitement (TM)
  tm = 0
  for j in range(3):
    for m in range(72):
      if j == 0:
        tm += x[m] * G / wk[0]
      elif j == 1:
        tm += (x[m] * G / wk[0]) + (x[m] * G / wk[1])
      elif j == 2:
        tm += (x[m] * G / wk[0]) + (x[m] * G / wk[1]) + (x[m] * G / wk[2])

  # Calcul du temps d'attente (TW)
  tw = 0
  for n in range(0, N):
    tw += x[n] * (G * C) / S

  # Calcul du temps de transfert (TB)
  tb = 0
  for j in range(3):
    for m in range(72):
      if j == 0:
        tb += x[m] * G / wk[0]
      elif j == 1:
        tb += (x[m] * G / wk[0]) + (x[m] * G / wk[1])
      elif j == 2:
        tb += (x[m] * G / wk[0]) + (x[m] * G / wk[1]) + (x[m] * G / wk[2])

  # Temps de complétion total
  te = tm + tw + tb
  return te

#     return np.sum(position * demandes_taches * capacites_serveurs)
def f2_energy(position):
  """
  Calcule la consommation d'énergie totale des serveurs MEC.

  Args:
    position: Un tableau contenant les valeurs des variables de décision.

  Returns:
    La consommation d'énergie totale (EC).
  """
  # Décodage des variables
  wnt = position[:72]  # Puissance allouée aux tâches
  atn = position[72:144]  # Type d'antenne utilisé pour chaque tâche

  # Calcul de la consommation d'énergie de base (ECB)
  ecb = 0
  for i in range(0, N):
    ecb += wnt[i] * rx

  # Calcul de la consommation d'énergie liée au front-haul (ECF)
  ecf = 0
  for n in range(0, N):
    for m in range(0, M):
      ecf += phi[atn[m]] * wnt[n] * rf

  # Calcul de la consommation d'énergie liée au backhaul (ECK)
  eck = 0
  eckm = eck
  for n in range(0, N):
    for m in range(0, M):
      eckm += phi[atn[m]]
    eck += (ZYS - eckm) * wnt[n] * rn

  # Consommation d'énergie totale
  ec = ecb + ecf + eck
  return ec

#     return np.max(np.sum(position * demandes_taches, axis=1))

def f3_variance(position):
  """
  Fonction objective pour le modèle d'équilibrage de charge.

  Args:
    position: Un tableau contenant les valeurs des variables de décision.

  Retourne:
    La variance de l'équilibrage de charge des serveurs MEC.
  """

  # Calcul de msu
  for n in range(0, nb_vm):
    msu_n += phi * size_vm
  msu = (1 / nb_vm) * msu_n

  # Calcul de la charge totale du système
  for n in range(0, N):
    msu_sum += msu
  ZYS = (1 / N) * msu_sum

  # Calcul de la variance de l'équilibrage de charge
  MSU = sum((msu - ZYS)**2) / N

  return MSU

## Implémentation de Firefly

### Paramètres de Firefly

In [ ]:
# Paramètres de l'algorithme Firefly
alpha = 0.5  # Coefficient d'absorption
beta = 0.2  # Coefficient de randomisation
gamma = 1.0  # Facteur d'atténuation

### Algorithme

In [ ]:
def fitness(position, w1, w2, w3):

  # Temps d'exécution
  temps = f1_time(position)

  # Consommation d'énergie
  energie = f2_energy(position)

  variance = f3_variance(position)

  # Fonction multiobjectives pondérée (c'est comme ça qu'on rend firefly multi-objectif)
  return w1 * energie + w2 * temps + w3 * variance


constraints = [{'type': 'eq', 'fun': c1_nbVM},
               {'type': 'ineq', 'fun': c2_ressources}]

In [ ]:
def validation(position, n_serveurs, demandes_taches, capacites_serveurs, nb_vm_inactives):
  # Contrainte de ressources
  for i in range(n_serveurs):
    if np.sum(position[:, i] * demandes_taches) > capacites_serveurs[i]:
      return False

  # Contrainte de VM
  nb_vm_par_serveur = np.sum(position, axis=0)
  for i in range(n_serveurs):
    if nb_vm_par_serveur[i] > nb_vm_inactives[i]:
      return False

  return True


In [ ]:
def firefly_algorithm(
  n_taches,
  n_serveurs,
  capacites_serveurs,
  demandes_taches,
  nb_vm_inactives,
  w1=1,
  w2=1,
  w3=1,
  alpha,
  beta,
  gamma,
  iterations
  ):

  """
  Fonction implémentant l'algorithme Firefly pour l'optimisation de l'allocation de VM.

  Args:
    n_taches: Nombre de tâches.
    n_serveurs: Nombre de serveurs.
    capacites_serveurs: Capacités des serveurs.
    demandes_taches: Demandes des tâches.
    nb_vm_inactives: Nombre de VM inactives par serveur.
    w1: Poids de la consommation d'énergie.
    w2: Poids du temps d'exécution.
    w3: Poids du déséquilibre de charge.
    alpha: Coefficient d'absorption.
    beta: Coefficient de randomisation.
    gamma: Facteur d'atténuation.
    iterations: Nombre d'itérations.

  Returns:
    Meilleure position des lucioles et la meilleure valeur de la fonction d'objectif.
  """

  # Initialisation des positions des lucioles
  positions = np.random.randint(0, 2, size=(n_taches, n_serveurs))

  # Initialisation des intensités des lucioles
  intensities = np.zeros(n_taches)

  for iteration in range(iterations):

    # Mise à jour des intensités des lucioles
    intensities = np.zeros(n_taches)
    for i in range(n_taches):
      if validation(positions[i], n_serveurs, demandes_taches, capacites_serveurs, nb_vm_inactives):
        intensities[i] = fitness(positions[i])

    # Mise à jour des positions des lucioles
    for i in range(n_taches):
      if validation(positions[i]):
        for j in range(n_serveurs):
          for k in range(n_taches):
            if k != i and validation(positions[i]):
              # Calcul de la distance entre les lucioles i et k
              distance = np.linalg.norm(positions[i] - positions[k])

              # Attraction vers la luciole la plus brillante
              attraction = alpha * intensities[k] * np.exp(-gamma * distance**2)

              # Perturbation aléatoire
              perturbation = beta * np.random.rand()

              # Mise à jour de la position de la luciole
              positions[i][j] += attraction + perturbation

  # Evaluation de la meilleure solution
  best_position = positions[np.argmin(intensities)]
  best_fitness = fitness(best_position, w1, w2, w3)

  return best_position, best_fitness

